In [4]:
#%pip install beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io

- Buscar os links para a tese no Stardog
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Stardog

In [6]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [7]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [ ]:
#Saprql query
query1 = """
SELECT ?thesis ?link
WHERE {
  ?thesis <http://purl.org/dc/terms/publisher> <tag:stardog:api:USP>.
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/repository> ?link.  
}
"""
#Cypher query 
query2 = """
MATCH  (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
RETURN  t["uri"] as Thesis, t["repository"] as Repositories 
LIMIT 25
"""

csv_results = conn.select(query1, content_type='text/csv')
thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
thesis_abstract

#2213 / 2208

thesis  \
0     tag:stardog:api:pequeninas_pobres_e_miseraveis...   
1     tag:stardog:api:partus_sequitur_ventrem_reprod...   
2     tag:stardog:api:o_abolicionismo_escolar_reflex...   
3     tag:stardog:api:a_maconaria_e_o_processo_da_ab...   
4     tag:stardog:api:garcas_nos_ceus_cafezais_na_te...   
...                                                 ...   
2208  tag:stardog:api:circulos_de_coca_e_fumaca_enco...   
2209  tag:stardog:api:trabajando_con_huacos_curander...   
2210  tag:stardog:api:producao_de_imagens_na_perfoma...   
2211  tag:stardog:api:niyayou_antagonismo_e_alianca_...   
2212  tag:stardog:api:dos_rios_e_suas_margens_narrat...   

                                                   link  
0     https://www.teses.usp.br/teses/disponiveis/8/8...  
1     https://www.teses.usp.br/teses/disponiveis/8/8...  
2     http://www.teses.usp.br/teses/disponiveis/48/4...  
3     http://www.teses.usp.br/teses/disponiveis/8/81...  
4     https://www.teses.usp.br/teses/disponiveis/8/8...  
...                                                 ...  
2208  http://www.teses.usp.br/teses/disponiveis/8/81...  
2209  https://www.teses.usp.br/teses/disponiveis/71/...  
2210  https://www.teses.usp.br/teses/disponiveis/27/...  
2211  http://www.teses.usp.br/teses/disponiveis/8/81...  
2212  https://www.teses.usp.br/teses/disponiveis/8/8...  

[2213 rows x 2 columns]

In [29]:
thesis_abstract['link'][0]

'https://www.teses.usp.br/teses/disponiveis/8/8138/tde-25112022-164104/'

In [42]:
url0 = ('https://bdtd.ibict.br/vufind/Search/Results?lookfor=Slave&type=AllFields&limit=20')
url = ('https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/pt-br.php')
url

'https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/pt-br.php'

In [19]:
import certifi
import ssl
import urllib3

In [49]:
#Fazer requisição e parsear o arquivo html
f = requests.get(url, verify=False).text#, verify=False) 
soup = bs(f, "html.parser")

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [50]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="pt-br" xml:lang="pt-br" xmlns="http://www.w3.org/1999/xhtml">
<head>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TK26HK5H');
  </script>
<!-- End Google Tag Manager -->
<title>Riqueza e escravidão no nordeste paulista: Batatais, 1851-1887</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<meta content="index, follow" name="robots"/>
<meta content="O presente trabalho dedica-se ao estudo da economia e da demografia da escr

In [55]:
#Coletando link para as teses
for doc in soup.find_all('a', href=True):
    if doc['href'].endswith('.pdf'):
        path = doc['href']
        prefix_uni = 'https://www.teses.usp.br'
        link = prefix_uni + path

link

'https://www.teses.usp.br/teses/disponiveis/8/8137/tde-16072007-105821/publico/Garavazo_Juliana_mestrado.pdf'

In [58]:
#fazer download do arquivo
def download_file(url, local_filename):
    # NOTE the stream=True parameter below
    r = requests.get(url, verify=False, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return local_filename

In [63]:
download_file(link, 'garavazo.pdf')

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.teses.usp.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'garavazo.pdf'

In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details